In [2]:
def main():
    print("hello world")
    
if __name__ == "__main__":
    main()

hello world


In [3]:
import sklearn
import numpy as np
import csv
import xlrd
import scipy.io
import pandas as pd
from pandas import ExcelFile
from pandas import ExcelWriter
import os
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC

In [4]:
data_dir = 'C:/Users/Zman/Desktop/Jiook/code_embarc/data/connectome_fmri'
home_dir = 'C:/Users/Zman/Desktop/Jiook/Zachs_Repo'
cwd = os.getcwd()
print(cwd)

C:\Users\Zman\Desktop\Jiook\Zachs_Repo


In [40]:
fname_1 = 'all_fmri_pat.xlsx'
fname_2 = 'real_names.xlsx'

def open_excel(fname):
    xl_workbook = xlrd.open_workbook(fname)
    xl_sheet = xl_workbook.sheet_by_index(0)
    return xl_sheet

def get_list():
    os.chdir(home_dir)
    names = []
    full_names = []
    labels = []
    temp = []
    mats = []
    xl_sheet = open_excel(fname_1) # excel sheet with abreviate patients names and whether they completed study + responded
    xl_sheet_2 = open_excel(fname_2)
    print(xl_sheet.cell(1, 1).value)
    print(xl_sheet.ncols, xl_sheet.nrows)
    for i in range(xl_sheet.nrows): # add patients who have completed the study to names and labels
        if (xl_sheet.cell(i, 1).value == 0 or xl_sheet.cell(i, 1).value == 1):
            names.append(xl_sheet.cell_value(i, 0))
    xl_sheet2 = open_excel(fname_2) # excel sheet with full names of patients - important to access their mats
    print(xl_sheet2.cell(2, 0).value[0:6])
    full_names = [xl_sheet2.cell(i, 0).value for i in range(xl_sheet2.nrows) if (xl_sheet2.cell(i, 0).value[0:6] in names)]
    names = list(name[0:6] for name in full_names)
    labels = list((xl_sheet.cell(i, 3).value for i in range(xl_sheet.nrows) if (xl_sheet.cell(i, 0).value in names))) 
    os.chdir(data_dir)
    print(labels)
    mats = []
    for name in full_names:
        with open("correlation_matrices_" + name + ".csv") as csvfile:
            mats.append(list(csv.reader(csvfile)))
    mats = np.reshape(mats, 86 * 39 * 39)
    mats = list([float(item) for item in mats])
    mats = np.array(mats)
    mats = np.reshape(mats, (86, 39 * 39))

    return mats, labels

get_list()    


1.0
1060 137
CU0009
[1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0]


(array([[1.        , 0.92325091, 0.6673523 , ..., 0.55062259, 0.85855175,
         1.        ],
        [1.        , 0.9345754 , 0.77149296, ..., 0.83628618, 0.83868409,
         1.        ],
        [1.        , 0.90808375, 0.66764748, ..., 0.61966   , 0.87388941,
         1.        ],
        ...,
        [1.        , 0.79768977, 0.52043944, ..., 0.50779247, 0.69541395,
         1.        ],
        [1.        , 0.77458412, 0.55936137, ..., 0.21997195, 0.54303869,
         1.        ],
        [1.        , 0.52452542, 0.0930237 , ..., 0.52175212, 0.26140053,
         1.        ]]),
 [1.0,
  1.0,
  1.0,
  1.0,
  0.0,
  1.0,
  1.0,
  1.0,
  0.0,
  1.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  1.0,
  0.0,
  0.0,
  0.0,
  0.0,
  1.0,
  0.0,
  0.0,
  0.0,
  1.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  1.0,
  1.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  1.0,
  0.0,
  0.0,
  1.0,
  1.0,
  1.0,
  0.0,
  1.0,
  0.0,
  0.0,
  1.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0

In [58]:
class Perceptron():
    def __init__(self, data_size, threshold=100, learning_rate=0.01):
        self.threshold = threshold
        self.learning_rate = learning_rate
        self.weights = np.zeros(data_size + 1)
        
    def predict(self, inputs):
        pred = np.dot(self.weights[1:], inputs) + self.weights[0]
        return float(pred > 0)
    
    def train(self, train_data, labels):
            
        for _ in range(self.threshold):
            for inputs, label in zip(train_data, labels):
                pred = self.predict(inputs)
                self.weights[1:] += self.learning_rate * (label - pred) * inputs
                self.weights[0] += self.learning_rate * (label - pred)
                
    def get_acc(self, preds, labels):
        count = 0
        correct = []
        data = zip(preds, labels)
        for d in data:
            if (d[0] == d[1]):
                count+=1
                correct.append(1.0)
            else:
                correct.append(0.0)
        print('preds are ', preds, ' and labels are ', labels)
        return correct


In [62]:
def simple_classification(mats, labels):
    train_preds = []
    test_preds = []
    X_train, X_test, y_train, y_test = train_test_split(mats, labels, test_size=0.33, random_state=42)
    clf = Perceptron(len(mats[0]))
    clf.train(X_train, y_train)
    for data in X_train:
        train_preds.append(clf.predict(data))
    for data in X_test:
        test_preds.append(clf.predict(data))
    train_correct = clf.get_acc(train_preds, y_train)
    print('accuracy is ', clf.get_acc(test_preds, y_test))
    
    return X_train, train_correct
    # x_train, predictions about x_train, labels
mats, labels = get_list()
train_mats, correct = simple_classification(mats, labels)
simple_classification(train_mats, correct)
print('the correct')

1.0
1060 137
CU0009
[1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0]
preds are  [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]  and labels are  [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0,

(array([[1.        , 0.94145763, 0.4165461 , ..., 0.29778463, 0.80728466,
         1.        ],
        [1.        , 0.85704928, 0.64149351, ..., 0.63217454, 0.62898094,
         1.        ],
        [1.        , 0.90808375, 0.66764748, ..., 0.61966   , 0.87388941,
         1.        ],
        ...,
        [1.        , 0.77458412, 0.55936137, ..., 0.21997195, 0.54303869,
         1.        ],
        [1.        , 0.95666661, 0.64142572, ..., 0.54578102, 0.83465806,
         1.        ],
        [1.        , 0.89841377, 0.51535408, ..., 0.55599611, 0.81924791,
         1.        ]]),
 [1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0])